<a href="https://colab.research.google.com/github/Tealbox-Digital/KodeKloud_rolling_window_PoCs/blob/main/PoC_enrollment_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run This To Install The Libraries and Connect To The Big Query and Allow All The Permissions Asked In The Dialog Box That Operns After Running The Cell

In [16]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project='kk-data-analytics')

import pandas as pd

#1. A simple Step-By_Step Process to get the Purchases for Multiple Rolling Window range

a. Enter the Date Range you want to get the Data for

----------------------------------Caution--------------------------------

the end date is not included in the analysis, for example if you want the data from 1 to 7 than put the start date as 1 and the end date as 8, here 8 wont be included in the analysis, so keep this is mind while entering the date range

In [ ]:
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")

Enter start date (YYYY-MM-DD): 2025-07-15
Enter end date (YYYY-MM-DD): 2025-07-22


b. Run the below two cells to get an idea about what are the columns in the leads table and how many entries are present in the time range you selected

ASSUMPTIONS TAKEN::

We are excludiung the values where the email of the leads in not present

In [ ]:
query = f"""
WITH enriched_leads AS (
  SELECT *,
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,
    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
  AND useremail IS NOT NULL
  AND useremail != ''
)

SELECT DISTINCT
  useremail,
  lead_submission_date,
  lead_utm_campaign_first,
  lead_utm_campaign_latest,
  utm_source_first,
  utm_source_latest,
  is_new_addition,
  CASE
    WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
    WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
    WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
    ELSE geography_first
  END AS Geography
FROM enriched_leads
"""
df_filters = client.query(query).to_dataframe()
df_filters.head(25)

In [ ]:
df_filters.shape

NameError: name 'df_filters' is not defined

In [ ]:
df_filters.shape    #this tells us (no. of rows, no. of columns)

(21215, 8)

* GETTING ALL THE AVAILABLE UNIQUE CAMPAIGNS AND OTHER VALUES AVAILABLE IN THE DATA, SO THAT USER CAN SELECT WHAT ALL THE FILTERS HE/SHE WANTS TO APPLY ON THE DATA

* SELECT FROM THE LIST BELOW, ALSO TO SELECT MULTIPLE OPTIONS, CLICK ON THE OPTIONS WHILE PRESSING THE "ctrl" KEY

* ALSO HERE IF YOU SEE 'NONE' OR '(DIRECT)' IN THE OPTIONS FOR THE lead_utm_campaign_first THEN IT MEANS THEY ARE ORGANIC LEADS

* IF YOU SEE BOTH OF THEM SELECT BOTH OF THEM FOR THE ANALYSIS RELATED TO THE ORGANIC LEADS

* IN THE 'is_new_addition' COLUMN '1' REPRESMTS THAT IF YOU WANT TO NEW LEADS AND '0' FOR THE LEADS THAT ARE NOT NEW


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Dictionary to store widgets for each column
selectors = {}

# Create and display widgets
for col in df_filters.columns:
    # Get unique values including None
    unique_vals = df_filters[col].unique().tolist()

    # Sort non-None values and add None at the beginning if it exists
    if None in unique_vals:
        unique_vals.remove(None)
        unique_vals = sorted(unique_vals)
        unique_vals.insert(0, None) # Add None at the beginning
    else:
        unique_vals = sorted(unique_vals)

    multi_select = widgets.SelectMultiple(
        options=unique_vals,
        description=col,
        rows=min(10, len(unique_vals)),
        layout=widgets.Layout(width='60%')
    )
    selectors[col] = multi_select
    display(multi_select)


SelectMultiple(description='useremail', layout=Layout(width='60%'), options=('0002ashish.rana@gmail.com', '000…

SelectMultiple(description='lead_submission_date', layout=Layout(width='60%'), options=(Timestamp('2025-01-01 …

SelectMultiple(description='lead_utm_campaign_first', layout=Layout(width='60%'), options=(None, '1st_post_-_f…

SelectMultiple(description='lead_utm_campaign_latest', layout=Layout(width='60%'), options=(None, '10178948-Sp…

SelectMultiple(description='utm_source_first', layout=Layout(width='60%'), options=(None, '(direct)', 'DDD Mel…

SelectMultiple(description='utm_source_latest', layout=Layout(width='60%'), options=(None, 'Home', 'HubSpot', …

SelectMultiple(description='is_new_addition', layout=Layout(width='60%'), options=(0, 1), rows=2, value=())

SelectMultiple(description='Geography', layout=Layout(width='60%'), options=('India', 'ROTW', 'TBU', 'UK', 'US…

Run the below table to know what filters you have selected

In [ ]:
from os import PRIO_PGRP
# Create a dictionary to hold selected values
selected_filters = {}

# Loop through each widget and get the selected values
for col, widget in selectors.items():
    selected_filters[col] = list(widget.value)  # Convert from tuple to list

# Example: print them all
for col, selected_values in selected_filters.items():
    print(f"{col}: {selected_values}")
# print(selected_filters['lead_utm_campaign_first'])

useremail: []
lead_submission_date: []
lead_utm_campaign_first: ["KK_GDN_IN_B2C-Generic-DevOps_PM_NB_PROS_NOA_APR'25-TB", 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', "KK_GDN_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR'25-TB", 'KK_GDN_IN_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', "KK_GDN_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_30_APR'25-TB", 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', "KK_GDN_ROTW_B2C-Generic-DevOps_PM_NB_PROS_NOA_30_APR'25-TB", 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', "KK_GDN_ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_30_APR'25-TB", 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', "KK_GDN_US/IN/ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR'25-TB", "KK_GDN_US/IN/ROTW_B2C-Generic-DevOps_PM_NB_PROS_NOA_APR'25-TB", "KK_GDN_US/IN/ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR'25-TB", 'KK_GDN_US_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_US_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', "KK_GDN_US_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR'25-TB

Here we are converting the filters into the sql query

Running the below cell will tell you how many total leads are there and how many of then are unique

In [ ]:
# 1) Build your dynamic filter string exactly as before:
selected_filters = { col: list(widget.value) for col, widget in selectors.items() }

where_clauses = []
for col, values in selected_filters.items():
    if not values:
        continue

    sql_literals = []
    include_null_check = False

    for v in values:
        if v is None:
            include_null_check = True
        elif isinstance(v, str):
            safe = v.replace("'", "\\'")
            sql_literals.append(f"'{safe}'")
        else:
            sql_literals.append(str(v))

    column_clauses = []
    if sql_literals:
        column_clauses.append(f"{col} IN ({', '.join(sql_literals)})")

    if include_null_check:
        column_clauses.append(f"{col} IS NULL")

    if column_clauses:
        where_clauses.append(f"({' OR '.join(column_clauses)})")


filter_sql = " AND ".join(where_clauses) if where_clauses else "1=1"
print(filter_sql)
# 2) Plug into this two‐step CTE query:
query = f"""
    WITH leads_base AS (
  SELECT

    useremail,
    lead_submission_date,
    lead_utm_campaign_first,
    lead_utm_campaign_latest,
    utm_source_first,
    utm_source_latest,
    is_new_addition,
    DATE(lead_submission_date) AS lead_date,

    -- STEP 1: compute the two parts separately
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,

    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last

  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE
     lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
     and
    useremail IS NOT NULL
    AND useremail != ''
),

enriched_leads AS (
  SELECT
    *,
    -- STEP 2: now you can reference geography_first/last
    CASE
      WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
      WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
      WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
      ELSE geography_first
    END AS Geography
  FROM leads_base
),
filtered_leads AS (
SELECT DISTINCT
 *
FROM enriched_leads
WHERE {filter_sql}
),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    )


SELECT
  COUNT(*)               AS total_leads,
  COUNT(DISTINCT useremail) AS unique_leads

FROM filtered_leads
"""
# print(query)
# 3) Execute
df_leads = client.query(query).to_dataframe()


(lead_utm_campaign_first IN ('KK_GDN_IN_B2C-Generic-DevOps_PM_NB_PROS_NOA_APR\'25-TB', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR\'25-TB', 'KK_GDN_IN_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_30_APR\'25-TB', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Generic-DevOps_PM_NB_PROS_NOA_30_APR\'25-TB', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_30_APR\'25-TB', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_GDN_US/IN/ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR\'25-TB', 'KK_GDN_US/IN/ROTW_B2C-Generic-DevOps_PM_NB_PROS_NOA_APR\'25-TB', 'KK_GDN_US/IN/ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR\'25-TB', 'KK_GDN_US_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_US_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_US_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR\'25-TB', 'KK_GDN_US_B2C-Sale-Camp

In [ ]:
df_leads.head()

,total_leads,unique_leads
0,17000,16940


Now we will get the enrollment data in the rolling window, just set up the rolling window values in daya according to what you want

You can select multiple windows at the same time just put them in the comma separated way like below:

windows = [7, 14, 28, 30, 60, 90, 120]

ASSUMPTIONS TAKEN::

we are removing the values where the email is not present of a user and also if the purchase subtotal is negative, we wont be including them in the analysis

In [ ]:
def compute_enrollment_counts(window_days, filter_sql):
    query = f"""
    WITH leads_base AS (
  SELECT

    useremail,
    lead_submission_date,
    lead_utm_campaign_first,
    lead_utm_campaign_latest,
    utm_source_first,
    utm_source_latest,
    is_new_addition,
    DATE(lead_submission_date) AS lead_date,

    -- STEP 1: compute the two parts separately
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,

    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last

  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE
     lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
     and
    useremail IS NOT NULL
    AND useremail != ''
),

enriched_leads AS (
  SELECT
    *,
    -- STEP 2: now you can reference geography_first/last
    CASE
      WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
      WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
      WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
      ELSE geography_first
    END AS Geography
  FROM leads_base
),
filtered_leads AS (
SELECT DISTINCT
 *
FROM enriched_leads
WHERE {filter_sql}
),

cohort_stats AS (
      SELECT COUNT(*) AS total_leads
  FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.useremail, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest

      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Course_enrollment_2025_v1_1` r
        ON r.email = fl.useremail
       AND DATE(r.created_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)


    )

    SELECT
      {window_days}             AS window_days,
      (SELECT total_leads FROM cohort_stats) AS total_leads,
  COUNT(*) AS enrollment_count,
  SAFE_DIVIDE(COUNT(*),(SELECT total_leads FROM cohort_stats))*100 AS enrollment_rate

FROM purchases_window pw
CROSS JOIN cohort_stats cs;

    """
    return client.query(query).to_dataframe()


# ─── 3) Run for each rolling window and combine ────────────────────────────────
windows = [7, 14, 30, 60, 120]
results = []
for w in windows:
    df = compute_enrollment_counts(w, filter_sql)
    results.append(df)

final_df = pd.concat(results, ignore_index=True)
display(final_df)

# ─── OPTIONAL: EXPORT ─────────────────────────────────────────────────────────
final_df.to_csv("rolling_window_enrollment_counts.csv", index=False)
print("✅ CSV exported to rolling_window_enrollment_counts.csv")

,window_days,total_leads,enrollment_count,enrollment_rate
0,7,17000,6670,39.235294
1,14,17000,6782,39.894118
2,30,17000,6905,40.617647
3,60,17000,7004,41.200000
4,120,17000,7074,41.611765


✅ CSV exported to rolling_window_enrollment_counts.csv


Now instead of getting the combined enrollment data for all the 'lead_utm_campaign_first' selected earlier, if you want to get their individual enrollments in the same date range run the below command

if you want to run it for any other filter just change the column name here: "for item in selected_filters['lead_utm_campaign_first']:"

and select those columns from the drop down filters above and run the cell below

In [ ]:
def compute_purchase_counts(window_days, filter_value):
    # Ensure filter_value is a string with single quotes around it and handle escaping
    if isinstance(filter_value, str):
        safe = filter_value.replace("'", r"\'")
        # now safe looks like: KK_SEM_IN_B2C‑Brand_PM_B_PROS_NOA_SEP\'24
        filter_sql = f"('{safe}')" # Modified to format for the IN clause
    else:
        filter_sql = f"({str(filter_value)})" # Modified to format for the IN clause

    query = f"""
    WITH leads_base AS (
  SELECT

    useremail,
    lead_submission_date,
    lead_utm_campaign_first,
    lead_utm_campaign_latest,
    utm_source_first,
    utm_source_latest,
    is_new_addition,
    DATE(lead_submission_date) AS lead_date,

    -- STEP 1: compute the two parts separately
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,

    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last

  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE
     lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
     and
    useremail IS NOT NULL
    AND useremail != ''
),

enriched_leads AS (
  SELECT
    *,
    -- STEP 2: now you can reference geography_first/last
    CASE
      WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
      WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
      WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
      ELSE geography_first
    END AS Geography
  FROM leads_base
),
filtered_leads AS (
SELECT DISTINCT
 *
FROM enriched_leads
WHERE lead_utm_campaign_first IN {filter_sql}
),

cohort_stats AS (
      SELECT COUNT(*) AS total_leads
  FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.useremail, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest

      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Course_enrollment_2025_v1_1` r
        ON r.email = fl.useremail
       AND DATE(r.created_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)


    )

    SELECT
      {window_days}             AS window_days,
      (SELECT total_leads FROM cohort_stats) AS total_leads,
  COUNT(*) AS enrollment_count,
  SAFE_DIVIDE(COUNT(*),(SELECT total_leads FROM cohort_stats))*100 AS enrollment_rate

FROM purchases_window pw
CROSS JOIN cohort_stats cs;
    """
    return client.query(query).to_dataframe()

# # ─── EXECUTE FOR MULTIPLE WINDOWS ──────────────────────────────────────────────
# windows = [7, 14, 30, 60, 90, 120]
# results = []
# for w in windows:
#     df = compute_purchase_counts(w, filter_sql)
#     results.append(df)

# final_df = pd.concat(results, ignore_index=True)
# display(final_df)

# # ─── OPTIONAL: EXPORT TO CSV ───────────────────────────────────────────────────
# final_df.to_csv("rolling_window_purchase_counts.csv", index=False)
# print("✅ CSV exported to rolling_window_purchase_counts.csv")
print(f"Between dates {start_date} and {end_date}, excluding {end_date}")
i=1
print(len(selected_filters['lead_utm_campaign_first']))
for item in selected_filters['lead_utm_campaign_first']:
  windows = [7, 14, 21, 28]
  results = []
  for w in windows:
    df = compute_purchase_counts(w, item)
    results.append(df)

  final_df = pd.concat(results, ignore_index=True)
  print(f"{i}----- {item}\n")
  i=i+1
  display(final_df)

Between dates 2025-01-01 and 2025-02-01, excluding 2025-02-01
53
1----- KK_B2C_IN_DG_DevOpsTools



,window_days,total_leads,enrollment_count,enrollment_rate
0,7,2,0,0.0
1,14,2,0,0.0
2,21,2,0,0.0
3,28,2,0,0.0


2----- KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA



,window_days,total_leads,enrollment_count,enrollment_rate
0,7,3,2,66.666667
1,14,3,2,66.666667
2,21,3,2,66.666667
3,28,3,2,66.666667


3----- KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA



...

,window_days,total_leads,enrollment_count,enrollment_rate
0,7,1,1,100.0
1,14,1,1,100.0
2,21,1,1,100.0
3,28,1,1,100.0


52----- KK_YT_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2



,window_days,total_leads,enrollment_count,enrollment_rate
0,7,1,1,100.0
1,14,1,1,100.0
2,21,1,1,100.0
3,28,1,1,100.0


53----- KK_YT_US_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA



,window_days,total_leads,enrollment_count,enrollment_rate
0,7,1,0,0.0
1,14,1,0,0.0
2,21,1,0,0.0
3,28,1,0,0.0


# 2. A quick way to do when you want a specific campaigns, this will give you the purchases in the rolling window directly

enter the date, remeber that the end date would not be included in the analysis so you may have to put one day more than the date range you want to analyze

In [ ]:
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")

Enter start date (YYYY-MM-DD): 2025-01-01
Enter end date (YYYY-MM-DD): 2025-07-18


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

# run your distinct‐campaigns query (assumes start_date/end_date & 'client' already defined)
query = f"""
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE lead_utm_campaign_first LIKE 'KK%'  AND lead_utm_campaign_first LIKE '%Cloud%' AND lead_utm_campaign_first NOT LIKE '%FBA%'
    AND lead_submission_date
        BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}');
"""
df_new_filters = client.query(query).to_dataframe()

# prepare the selectors dict
selectors = {}

# 1) Campaigns multi‐select
campaign_options = df_new_filters['lead_utm_campaign_first'].tolist()
campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,    # pre‑select all campaigns by default
    description='Campaigns',
    rows=min(10, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

# 2) is_new_addition multi‐select
#    (you can pick 0, 1, both or neither)
new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                  # none selected by default → no filter on this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='30%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget


SelectMultiple(description='Campaigns', layout=Layout(width='60%'), options=(), rows=0, value=())

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='30%'), options=(0, 1), rows=2, value=())

run this to convert the selected options into the query

In [ ]:
selected_filters = { col: list(widget.value) for col, widget in selectors.items() }

where_clauses = []
for col, values in selected_filters.items():
    if not values:
        continue
    literals = []
    for v in values:
        if isinstance(v, str):
            safe = v.replace("'", "\\'")
            literals.append(f"'{safe}'")
        else:
            literals.append(str(v))
    where_clauses.append(f"{col} IN ({', '.join(literals)})")

filter_sql = " AND ".join(where_clauses) if where_clauses else "1=1"
print(filter_sql)

lead_utm_campaign_first IN ('KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_GDN_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR\'25-TB', 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB') AND is_new_addition IN (1)


enter the window range you want the output for.

FOR EXAMPLE: windows = [7,14,21,28]

This will give you combined enrollment data for all the campaigns together

In [ ]:
def compute_enrollment_counts(window_days, filter_sql):
    query = f"""
    WITH leads_base AS (
  SELECT

    useremail,
    lead_submission_date,
    lead_utm_campaign_first,
    lead_utm_campaign_latest,
    utm_source_first,
    utm_source_latest,
    is_new_addition,
    DATE(lead_submission_date) AS lead_date,

    -- STEP 1: compute the two parts separately
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,

    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last

  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE
     lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
     and
    useremail IS NOT NULL
    AND useremail != ''
),

enriched_leads AS (
  SELECT
    *,
    -- STEP 2: now you can reference geography_first/last
    CASE
      WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
      WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
      WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
      ELSE geography_first
    END AS Geography
  FROM leads_base
),
filtered_leads AS (
SELECT DISTINCT
 *
FROM enriched_leads
WHERE {filter_sql}
),

cohort_stats AS (
      SELECT COUNT(*) AS total_leads
  FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.useremail, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest

      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Course_enrollment_2025_v1_1` r
        ON r.email = fl.useremail
       AND DATE(r.created_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)


    )

    SELECT
      {window_days}             AS window_days,
      (SELECT total_leads FROM cohort_stats) AS total_leads,
  COUNT(*) AS enrollment_count,
  SAFE_DIVIDE(COUNT(*),(SELECT total_leads FROM cohort_stats))*100 AS enrollment_rate

FROM purchases_window pw
CROSS JOIN cohort_stats cs;

    """
    return client.query(query).to_dataframe()


# ─── 3) Run for each rolling window and combine ────────────────────────────────
windows = [7, 14, 30, 60, 120]
results = []
for w in windows:
    df = compute_enrollment_counts(w, filter_sql)
    results.append(df)

final_df = pd.concat(results, ignore_index=True)
display(final_df)

# ─── OPTIONAL: EXPORT ─────────────────────────────────────────────────────────
final_df.to_csv("rolling_window_enrollment_counts.csv", index=False)
print("✅ CSV exported to rolling_window_enrollment_counts.csv")

#3. Multiple date ranges with their specific rolling windows

Select the campaigns by applying the sql filters correctly

In [ ]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE lead_utm_campaign_first LIKE 'KK%'  AND lead_utm_campaign_first LIKE '%Cloud%'
  -- AND lead_utm_campaign_first NOT LIKE '%Sale%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17), …

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

See the options you have selected and verify them

In [ ]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', "KK_GDN_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR'25-TB", "KK_GDN_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_30_APR'25-TB", "KK_GDN_US/IN/ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR'25-TB", 'KK_FBA_IN_B2C-Cloud_CONV_CONV_NA_RTG_MoF - Purchase - TB', 'KK_FBA_IN_B2C-Cloud_CONV_CONV_NA_RTG_BoF - Purchase', 'KK_FBA_IN_B2C-Cloud_CONV_CONV_NA_RTG_BoF - Purchase - TB', 'KK_FBA_IN_B2C-Cloud_CONV_NA_PROS_MULT - Purchase - TB', 'KK_FBA_IN_B2C-Cloud_CONV_NA_PROS_MULT - Purchase', 'KK_FBA_ROTW_B2C-Cloud_CONV_NA_PROS_MULT - Purchase - TB', 'KK_FBA_ROTW_B2C-Cloud_CONV_NA_RTG_MoF - Purchase - TB', 'KK_FBA_US_B2C-Cloud_CONV_NA_PROS_MULT - Purchase - TB', 'KK_PMAX_IN_B2C-MarchTowardsCloud-Sale_CONV_B_PROS_MU

Specify the date ranges and rolling windows you want to analyze in the format of a Python list of dictionaries.

For example:

In [ ]:
date_windows_text = widgets.Textarea(
    value='[{"start": "2025-01-01", "end": "2025-07-07", "windows": [7, 14]}, {"start": "2025-02-01", "end": "2025-07-08", "windows": [7]}]',
    description='Date Ranges and Windows:',
    layout=Layout(width='80%', height='100px')
)

display(date_windows_text)

Textarea(value='[{"start": "2025-01-01", "end": "2025-07-07", "windows": [7, 14]}, {"start": "2025-02-01", "en…

Now run this cell to get the results for multiple date ranges and rolling windows

In [ ]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
def compute_purchase_counts(window_days, filter_value, start_date, end_date):
    # Ensure filter_value is a string with single quotes around it and handle escaping
    if isinstance(filter_value, str):
        safe = filter_value.replace("'", r"\'")
        # now safe looks like: KK_SEM_IN_B2C‑Brand_PM_B_PROS_NOA_SEP\'24
        filter_sql = f"('{safe}')" # Modified to format for the IN clause
    else:
        filter_sql = f"({str(filter_value)})" # Modified to format for the IN clause

    query = f"""
    WITH leads_base AS (
  SELECT

    useremail,
    lead_submission_date,
    lead_utm_campaign_first,
    lead_utm_campaign_latest,
    utm_source_first,
    utm_source_latest,
    is_new_addition,
    DATE(lead_submission_date) AS lead_date,

    -- STEP 1: compute the two parts separately
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,

    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last

  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE
     lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
     and
    useremail IS NOT NULL
    AND useremail != ''
),

enriched_leads AS (
  SELECT
    *,
    -- STEP 2: now you can reference geography_first/last
    CASE
      WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
      WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
      WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
      ELSE geography_first
    END AS Geography
  FROM leads_base
),
filtered_leads AS (
SELECT DISTINCT
 *
FROM enriched_leads
WHERE lead_utm_campaign_first IN {filter_sql}
),

cohort_stats AS (
      SELECT COUNT(*) AS total_leads
  FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.useremail, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest

      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Course_enrollment_2025_v1_1` r
        ON r.email = fl.useremail
       AND DATE(r.created_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)


    )

    SELECT
      {window_days}             AS window_days,
      (SELECT total_leads FROM cohort_stats) AS total_leads,
  COUNT(*) AS enrollment_count,
  SAFE_DIVIDE(COUNT(*),(SELECT total_leads FROM cohort_stats))*100 AS enrollment_rate

FROM purchases_window pw
CROSS JOIN cohort_stats cs;
    """
    return client.query(query).to_dataframe()



# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
def run_batches():
    # parse date_windows from textarea safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    selected_filters = get_selected_filters()

    i = 1
    grand_total = 0
    all_outputs = []   # collect ((start,end), campaign, df)

    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} (excluding {end_date}) ===")
        print(f"Campaign filters: {len(selected_filters['lead_utm_campaign_first'])}")

        for campaign in selected_filters['lead_utm_campaign_first']:
            dfs = [compute_purchase_counts(w, campaign, start_date, end_date) for w in windows]
            final_df = pd.concat(dfs, ignore_index=True)

            print(f"\n{i} ----- {campaign}")
            grand_total += final_df['total_leads'].iloc[0]
            print("Running total leads =", grand_total)
            display(final_df)

            all_outputs.append(((start_date, end_date), campaign, final_df.copy()))
            i += 1

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds all DataFrames.")



=== Between 2025-01-01 and 2025-07-07 (excluding 2025-07-07) ===
Campaign filters: 18

1 ----- KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB
Running total leads = 5439


,window_days,total_leads,enrollment_count,enrollment_rate
0,7,5439,1306,24.011767
1,14,5439,1318,24.232396



2 ----- KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB
Running total leads = 6796


,window_days,total_leads,enrollment_count,enrollment_rate
0,7,1357,103,7.590273
1,14,1357,106,7.811349



3 ----- KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB
Running total leads = 7021


...


35 ----- KK_FBA_US_B2C-Cloud_CONV_NA_PROS_MULT - Purchase - TB
Running total leads = 17756


,window_days,total_leads,enrollment_count,enrollment_rate
0,7,9,1,11.111111



36 ----- KK_PMAX_IN_B2C-MarchTowardsCloud-Sale_CONV_B_PROS_MULT_NA
Running total leads = 17757


,window_days,total_leads,enrollment_count,enrollment_rate
0,7,1,1,100.0



Done. `results` holds all DataFrames.


#4. For multiple dates ranges and combined of all campaigns and filters

In [22]:
date_windows_text = widgets.Textarea(
    value='[{"start": "2025-01-01", "end": "2025-07-07", "windows": [7, 14]}, {"start": "2025-02-01", "end": "2025-07-08", "windows": [7]}]',
    description='Date Ranges and Windows:',
    layout=Layout(width='80%', height='80px')
)

display(date_windows_text)

Textarea(value='[{"start": "2025-01-01", "end": "2025-07-07", "windows": [7, 14]}, {"start": "2025-02-01", "en…

In [23]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE lead_utm_campaign_first LIKE 'KK%'  AND lead_utm_campaign_first LIKE '%Cloud%'
  -- AND lead_utm_campaign_first NOT LIKE '%Sale%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17), …

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

In [24]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB'], 'is_new_addition': [1]}


In [25]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
# =========================================
# 4. Core query function (SQL body minimally altered)
# =========================================
def compute_purchase_counts(window_days, start_date, end_date, extra_where):
    query = f"""
    WITH leads_base AS (
  SELECT

    useremail,
    lead_submission_date,
    lead_utm_campaign_first,
    lead_utm_campaign_latest,
    utm_source_first,
    utm_source_latest,
    is_new_addition,
    DATE(lead_submission_date) AS lead_date,

    -- STEP 1: compute the two parts separately
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,

    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_')   THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_')   THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_')   THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW')
                                                             THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last

  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v3_1`
  WHERE
     lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
     and
    useremail IS NOT NULL
    AND useremail != ''
),

enriched_leads AS (
  SELECT
    *,
    -- STEP 2: now you can reference geography_first/last
    CASE
      WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
      WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
      WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
      ELSE geography_first
    END AS Geography
  FROM leads_base
),
filtered_leads AS (
SELECT DISTINCT
 *
FROM enriched_leads
WHERE 1=1 {extra_where}
),

cohort_stats AS (
      SELECT COUNT(*) AS total_leads
  FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.useremail, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest

      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Course_enrollment_2025_v1_1` r
        ON r.email = fl.useremail
       AND DATE(r.created_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)


    )

    SELECT
      {window_days}             AS window_days,
      (SELECT total_leads FROM cohort_stats) AS total_leads,
  COUNT(*) AS enrollment_count,
  SAFE_DIVIDE(COUNT(*),(SELECT total_leads FROM cohort_stats))*100 AS enrollment_rate

FROM purchases_window pw
CROSS JOIN cohort_stats cs;
    """
    return client.query(query).to_dataframe()



# =========================================
# 3. Helper to build WHERE fragments
# =========================================
def _to_sql_in(values):
    """Return a BigQuery-safe IN (...) string for a Python list."""
    if not values:
        return None
    if isinstance(values[0], (int, float)):
        inner = ",".join(str(v) for v in values)
    else:
        inner = ",".join("'" + str(v).replace("'", "''") + "'" for v in values)
    return f"({inner})"

def build_extra_where(filters_dict):
    """
    Turn a dict like:
        {'lead_utm_campaign_first': [...], 'is_new_addition': [0,1]}
    into ANDed SQL fragments.
    """
    clauses = []
    for col, vals in filters_dict.items():
        if vals:
            in_clause = _to_sql_in(vals)
            clauses.append(f"{col} IN {in_clause}")
    if clauses:
        return " AND " + " AND ".join(clauses)
    return ""     # no extra filtering


# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
# =========================================
# 5. Runner – combined filters (no per-campaign loop)
# =========================================
def run_batches():
    # parse the date_windows text safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    filters_dict = get_selected_filters()
    extra_where = build_extra_where(filters_dict)

    all_outputs = []
    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} ===")
        print("Filters applied:", filters_dict)
        print(extra_where)

        dfs = [
            compute_purchase_counts(w, start_date, end_date, extra_where)
            for w in windows
        ]
        final_df = pd.concat(dfs, ignore_index=True)

        display(final_df)

        all_outputs.append(((start_date, end_date), final_df.copy()))

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds each (date_range, df) tuple.")


=== Between 2025-01-01 and 2025-07-07 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB'], 'is_new_addition': [1]}
 AND lead_utm_campaign_first IN ('KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB','KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB','KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB','KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB','KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB','KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB') AND is_new_addition IN (1)


,window_days,total_leads,enrollment_count,enrollment_rate
0,7,7361,1462,19.861432
1,14,7361,1479,20.092379



=== Between 2025-02-01 and 2025-07-08 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB'], 'is_new_addition': [1]}
 AND lead_utm_campaign_first IN ('KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB','KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB','KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB','KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB','KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB','KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB') AND is_new_addition IN (1)


,window_days,total_leads,enrollment_count,enrollment_rate
0,7,7368,1463,19.856135



Done. `results` holds each (date_range, df) tuple.
